# Import functions

In [1]:
import os
import glob
#import WAPORWA modules
os.chdir(r'C:\Users\h.jayasekara\Downloads\modules\modules') #change to modules path
import WaPOR
import WA
from WA.pickle_basin import pickle_in,pickle_out  
import pandas as pd
from osgeo import gdal
import numpy as np
import osr
from WaPOR import GIS_functions as gis
import calendar

# f931d000c500d538478b92992be10702c08e8656b6b54d43d8eecfad9ee8edeee4b34903360c03ac


Your WaPOR API Token: f931d000c500d538478b92992be10702c08e8656b6b54d43d8eecfad9ee8edeee4b34903360c03ac


# Collect data from WAPOR

In [ ]:
WaPOR.API.version=2
catalog=WaPOR.API.getCatalog()

INPUT_FOLDER=r'C:\Users\h.jayasekara\Desktop\WaporData' #change to input path

#%% DEFINE Basin extent
### OPTION 1: Define minimum and maximum longitude (x) and latitude (y)
# xmin,ymin,xmax,ymax=[34.4,29.4,37.0,33.7]

### OPTION 2: Read from shapefile
BASIN_SHP = r'C:\Users\h.jayasekara\Desktop\WaporData\palasthine' #path to Basin shapefile
import shapefile
shape=shapefile.Reader(BASIN_SHP)
xmin,ymin,xmax,ymax=shape.bbox

start_date='2015-03-01'
end_date='2022-02-01'

# WaPOR.PCP_daily(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25])

# WaPOR.PCP_monthly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25])

# WaPOR.RET_monthly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.5,ymax+0.5],
#                   lonlim=[xmin-0.5,xmax+0.5])

# WaPOR.AET_monthly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

WaPOR.NPP_monthly(INPUT_FOLDER,Startdate=start_date,
                  Enddate=end_date,latlim=[ymin,ymax],
                  lonlim=[xmin,xmax],level=2)

# WaPOR.LCC_yearly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

# WaPOR.I_dekadal(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

# WaPOR.E_dekadal(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

Loading WaPOR catalog...

Download monthly WaPOR Reference Evapotranspiration data for the period 2015-03-01 till 2022-02-01
Loading WaPOR catalog...
Progress: |███████████████████████████████████████-----------| 79.5% Complete

# Reclassify LCC to LUWA

In [4]:
Main_dir=r"/home/Nafn/WA_data/WAPORWA/data/Litani/Main"
LCC_folder=r"/home/Nafn/WA_data/WAPORWA/data/Litani/Input/L2_LCC_A" #folder of WaPOR LCC GeoTIFFs
Global_GRaND_Reservoir=r"/home/Nafn/WA_data/WAPORWA/data/Global/GRanD/GRanD_reservoirs_v1_3.shp" #path to  GRanD reservoirs shapfile
WDPA=r"/home/Nafn/WA_data/WAPORWA/data/Global/WDPA/WDPA_CatIandII_17countries.shp" #path to WDPA shapefile
                        
Resrv_to_Lake=None #GeoTIFF of selected reservoirs which are natural lakes
Lake_to_Reserv=None  #GeoTIFF of selected natural lakes which are reservoirs

In [5]:
"""
Created on Thu Aug  8 13:42:44 2019

@author: ntr002
WAPOR LCC to LUWA

Note: start Jupyter notebook with "conda activate env"
"""
import glob
from WA.LCC_to_LUWA import Rasterize_shape_basin,Adjust_GRaND_reservoir,LCC_to_LUWA

### Get all yearly WaPOR LCC maps
LCC_fhs=glob.glob(os.path.join(LCC_folder,'*.tif'))
WaPOR_LCC=LCC_fhs[0] #template

### Create Reservoir raster map for basin
Dir_out=os.path.join(Main_dir,'Reservoir')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
Basin_Reservoir_tif=os.path.join(Dir_out,'Reservoir_basin.tif')

# adjust reservoir
if (Resrv_to_Lake!=None)&(Lake_to_Reserv!=None): #require 2 shapefiles to unreservoir and makereservoir           
    Adjust_GRaND_reservoir(Basin_Reservoir_tif,WaPOR_LCC,
                           Global_GRaND_Reservoir,Resrv_to_Lake,Lake_to_Reserv)    
else:                
    Rasterize_shape_basin(Global_GRaND_Reservoir,WaPOR_LCC,Basin_Reservoir_tif)
    

### Create Protected Area raster map for basin
Dir_out=os.path.join(Main_dir,'Protected')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
    
ProtectedArea_tif=os.path.join(Dir_out,'ProtectedArea_basin.tif')
Rasterize_shape_basin(WDPA,WaPOR_LCC,ProtectedArea_tif)

### Reclassify LCC to LUWA       
Dir_out=os.path.join(Main_dir,'data','luwa')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
for fh in LCC_fhs:
    LCC_to_LUWA(fh,Dir_out,ProtectedArea_tif,Basin_Reservoir_tif)           

Rasterising shapefile...
Rasterising shapefile...


Warning 1: Failed to fetch spatial reference on layer WDPA_CatIandII_17countries to build transformer, assuming matching coordinate systems.
Warning 1: Value '000000000000       0.000' of field WDPA_CatIandII_17countries.REP_M_AREA parsed incompletely to real 0.
Warning 1: Value '000000000000    3046.610' of field WDPA_CatIandII_17countries.GIS_M_AREA parsed incompletely to real 0.
Warning 1: Value '888709999927    3058.402' of field WDPA_CatIandII_17countries.REP_AREA parsed incompletely to real 888709999927.
Warning 1: Value '930119999837Not Applicab' of field WDPA_CatIandII_17countries.GIS_AREA parsed incompletely to real 930119999837.
Warning 1: Value '000000000000Designated' of field WDPA_CatIandII_17countries.NO_TK_AREA parsed incompletely to real 0.
Warning 1: Value 'Not Applicable' of field WDPA_CatIandII_17countries.GIS_AREA parsed incompletely to real 0.
Warning 1: Value 'Designated' of field WDPA_CatIandII_17countries.NO_TK_AREA parsed incompletely to real 0.
Warning 1: Valu

# Create monthly Interception

In [6]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep 11 16:42:09 2019

@author: ntr002
"""

Dekadal_I_folder=r"/home/Nafn/WA_data/WAPORWA/data/Litani/Input/L2_I_D"
start_date='2009-01-01'
end_date='2014-12-31'
ds_code='L2_E_D' 
output_folder=Dekadal_I_folder.replace('L2_E_D','L2_E_M')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)

In [7]:
#%%
#get list of dekadal rasters

input_fhs=glob.glob(os.path.join(Dekadal_I_folder,'*.tif'))
input_fhs

#Get month dates
month_dates=pd.date_range(start_date,end_date,freq='M')

#Get df avail 
WaPOR.API.version=2
df_avail=WaPOR.API.getAvailData(ds_code,time_range='{0},{1}'.format(start_date,end_date))

#aggregate monthly
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(input_fhs[0])
for date in month_dates:
    month_fhs=[]
    for in_fh in input_fhs:
        raster_id=os.path.split(in_fh)[-1].split('.tif')[0][-9:]
        raster_info=df_avail.loc[df_avail['raster_id']==raster_id]
        timestr=raster_info['DEKAD'].iloc[0]
        year=int(timestr[0:4])
        month=int(timestr[5:7])
        if (year==date.year)&(month==date.month):
            month_fhs.append(in_fh)
    SumArray=np.zeros((ysize,xsize),dtype=np.float32)
    for fh in month_fhs:
        Array=gis.OpenAsArray(fh,nan_values=True)
        SumArray+=Array
    out_fh=os.path.join(output_folder,'WaPOR_E_{:04d}{:02d}.tif'.format(date.year,date.month))    
    gis.CreateGeoTiff(out_fh, SumArray, driver, NDV, xsize, ysize, GeoT, Projection)

Loading WaPOR catalog...


# Create Monthly Rainy Days

In [8]:
"""
Adapted from watools (by T.Hessels)
"""

###
Data_Path_P=r"/home/Nafn/WA_data/WAPORWA/data/Litani/Input/L1_PCP_E"
start_date='2009-01-01'
end_date='2014-12-31'

Data_Path_RD = Data_Path_P.replace('L1_PCP_E','Rainy_Days')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)

In [9]:
if not os.path.exists(Data_Path_RD):
    os.mkdir(Data_Path_RD)
Dates = pd.date_range(start_date, end_date, freq ='MS')
os.chdir(Data_Path_P)

for Date in Dates:
    # Define the year and month and amount of days in month
    year = Date.year
    month = Date.month
    daysinmonth = calendar.monthrange(year, month)[1]

    # Set the third (time) dimension of array starting at 0
    i = 0

    # Find all files of that month
    files = glob.glob('*daily_%d.%02d.*.tif' %(year, month))

    # Check if the amount of files corresponds with the amount of days in month
    if len(files) is not daysinmonth:
        print('ERROR: Not all Rainfall days for month %d and year %d are downloaded'  %(month, year))

    # Loop over the days and store data in raster
    for File in files:
        dir_file = os.path.join(Data_Path_P, File)

        # Get array information and create empty numpy array for daily rainfall when looping the first file
        if File == files[0]:

            # Open geolocation info and define projection
            driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(dir_file)

            # Create empty array for the whole month
            P_Daily = np.zeros([daysinmonth,ysize, xsize],dtype=np.float32)

        # Open data and put the data in 3D array
        Data = gis.OpenAsArray(dir_file,nan_values=True)

        # Remove the weird numbers
        Data[Data<0] = 0

        # Add the precipitation to the monthly cube
        P_Daily[i, :, :] = Data
        i += 1

    # Define a rainy day
    P_Daily[P_Daily > 0.201] = 1
    P_Daily[P_Daily != 1] = 0

    # Sum the amount of rainy days
    RD_one_month = np.nansum(P_Daily,0)

    # Define output name
    Outname = os.path.join(Data_Path_RD, 'Rainy_Days_NumOfDays_monthly_%d.%02d.01.tif' %(year, month))

    # Save tiff file
    gis.CreateGeoTiff(Outname, RD_one_month, driver, NDV, xsize, ysize, GeoT, Projection)
    
